In [89]:
import mysql.connector
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [77]:
with open('../sql_passkey.txt', 'r') as file: 
    pswd= file.read()

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=pswd,
    database="recommendation_system"
)
cursor = connection.cursor()

In [78]:
query = "SELECT * FROM movies"
cursor.execute(query)
movie_details= cursor.fetchall()

In [79]:
cols= ['movie_id','title', 'year', 'genres', 'ratings', 'certificate', 'directors', 'cast', 'description', 'runtime']

df= pd.DataFrame(movie_details, columns= cols)
df

,movie_id,title,year,genres,ratings,certificate,directors,cast,description,runtime
0,1,Escape to Grizzly Mountain (2000),2000,"Drama, Family, Sci-Fi",5.8,G,Anthony Dalesandro,"Dan Haggerty, Jan-Michael Vincent, Miko Hughes...",Escape to Grizzly Mountain: Directed by Anthon...,95.0
1,2,Chain of Command (2000),2000,"Action, Adventure, Thriller",4.6,R,John Terlesky,"Roy Scheider, Patrick Muldoon, Maria Conchita ...",Chain of Command: Directed by John Terlesky. W...,96.0
2,3,101 Ways (The Things a Girl Will Do to Keep He...,2000,Comedy,5.2,None,Jennifer B. Katz,"Wendy Hoopes, Jayce Bartok, Rick Bean, P.J. Br...",101 Ways (The Things a Girl Will Do to Keep He...,NaN
3,4,Apartment Hunting (2000),2000,"Comedy, Romance",5.9,None,Bill Robertson,"Andrew Tarbet, Kari Matchett, Matt Gordon, Tra...",Apartment Hunting: Directed by Bill Robertson....,NaN
4,5,Brothers. Dogs. And God. (2000),2000,"Comedy, Drama",7.3,Not Rated,David Hickey,"Dale Dudley, Bob Fonseca, Raul, Spike Alexande...",Brothers. Dogs. And God.: Directed by David Hi...,108.0
...,...,...,...,...,...,...,...,...,...,...
1659,1925,Who Is Cletis Tout? (2001),2001,"Action, Comedy, Crime",6.3,R,Chris Ver Wiel,"Christian Slater, Tim Allen, Portia de Rossi, ...",Who Is Cletis Tout?: Directed by Chris Ver Wie...,92.0
1660,1926,Everybody Says I'm Fine! (2001),2001,"Drama, Fantasy, Romance",5.9,Unrated,Rahul Bose,"Rehaan Engineer, Koel Purie, Rahul Bose, Pooja...",Everybody Says I'm Fine!: Directed by Rahul Bo...,103.0
1661,1927,Malunde (2001),2001,Drama,6.4,None,Stefanie Sycholt,"Ian Roberts, Kagiso Mtetwa, Winston Ntshona, M...",Malunde: Directed by Stefanie Sycholt. With Ia...,NaN
1662,1928,Inertia (2001),2001,Romance,5.5,None,Sean Garrity,"Jonas Chernick, Sarah Constible, Gordon Tanner...",Inertia: Directed by Sean Garrity. With Jonas ...,NaN


In [80]:
def preprocess_text(sentence):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sentence)
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and token.pos_ in ['NOUN', 'ADJ']]

    return ' '.join(tokens)


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1664 entries, 0 to 1663
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     1664 non-null   int64  
 1   title        1664 non-null   object 
 2   year         1664 non-null   int64  
 3   genres       1664 non-null   object 
 4   ratings      1664 non-null   float64
 5   certificate  1103 non-null   object 
 6   directors    1664 non-null   object 
 7   cast         1664 non-null   object 
 8   description  1664 non-null   object 
 9   runtime      1089 non-null   float64
dtypes: float64(2), int64(2), object(6)
memory usage: 130.1+ KB


In [85]:
df['keywords']= df['description'].apply(preprocess_text)

In [93]:
df

,movie_id,title,year,genres,ratings,certificate,directors,cast,description,runtime,keywords
0,1,Escape to Grizzly Mountain (2000),2000,"Drama, Family, Sci-Fi",5.8,G,Anthony Dalesandro,"Dan Haggerty, Jan-Michael Vincent, Miko Hughes...",Escape to Grizzly Mountain: Directed by Anthon...,95.0,young boy acceptance peers circus trainer bear...
1,2,Chain of Command (2000),2000,"Action, Adventure, Thriller",4.6,R,John Terlesky,"Roy Scheider, Patrick Muldoon, Maria Conchita ...",Chain of Command: Directed by John Terlesky. W...,96.0,chain agents briefcase control nuclear arsenal...
2,3,101 Ways (The Things a Girl Will Do to Keep He...,2000,Comedy,5.2,None,Jennifer B. Katz,"Wendy Hoopes, Jayce Bartok, Rick Bean, P.J. Br...",101 Ways (The Things a Girl Will Do to Keep He...,NaN,ways girl things girl epic adventures young wr...
3,4,Apartment Hunting (2000),2000,"Comedy, Romance",5.9,None,Bill Robertson,"Andrew Tarbet, Kari Matchett, Matt Gordon, Tra...",Apartment Hunting: Directed by Bill Robertson....,NaN,apartment hunting writer magazine fledgling ca...
4,5,Brothers. Dogs. And God. (2000),2000,"Comedy, Drama",7.3,Not Rated,David Hickey,"Dale Dudley, Bob Fonseca, Raul, Spike Alexande...",Brothers. Dogs. And God.: Directed by David Hi...,108.0,brothers dogs times man turn
...,...,...,...,...,...,...,...,...,...,...,...
1659,1925,Who Is Cletis Tout? (2001),2001,"Action, Comedy, Crime",6.3,R,Chris Ver Wiel,"Christian Slater, Tim Allen, Portia de Rossi, ...",Who Is Cletis Tout?: Directed by Chris Ver Wie...,92.0,comedy mistaken identities hitman terms movies...
1660,1926,Everybody Says I'm Fine! (2001),2001,"Drama, Fantasy, Romance",5.9,Unrated,Rahul Bose,"Rehaan Engineer, Koel Purie, Rahul Bose, Pooja...",Everybody Says I'm Fine!: Directed by Rahul Bo...,103.0,fine hair stylist minds hair information
1661,1927,Malunde (2001),2001,Drama,6.4,None,Stefanie Sycholt,"Ian Roberts, Kagiso Mtetwa, Winston Ntshona, M...",Malunde: Directed by Stefanie Sycholt. With Ia...,NaN,malunde post apartheid wiry year old crime str...
1662,1928,Inertia (2001),2001,Romance,5.5,None,Sean Garrity,"Jonas Chernick, Sarah Constible, Gordon Tanner...",Inertia: Directed by Sean Garrity. With Jonas ...,NaN,inertia love love love love mysterious germ wa...


In [96]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['keywords'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Define recommendation function
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = df['movie_id']
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Exclude the movie itself
    movie_indices = [i[0] for i in sim_scores]
    return df['Title'].iloc[movie_indices]

# Example usage
print(get_recommendations('Who Is Cletis Tout?'))

IndexError: index 1664 is out of bounds for axis 0 with size 1664